In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from numpyencoder import NumpyEncoder
import json

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import time
from codecarbon import track_emissions

# Load the COCO-2017 validation split into a FiftyOne dataset
#
# This will download the dataset from the web, if necessary
#
dataset = foz.load_zoo_dataset("coco-2017", split="validation")
#test17 = foz.load_zoo_dataset("coco-2017", split="test")

# Give the dataset a new name, and make it persistent so that you can
# work with it in future sessions
dataset.name = "coco-2017-sets"
dataset.persistent = True
#test17.name = "coco-2017-test-example"
#test17.persistent = True
# Visualize the in the App
#session = fo.launch_app(dataset)

Found annotations at '/Users/kristian/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [10.0s elapsed, 0s remaining, 527.2 samples/s]      
Dataset 'coco-2017-validation' created


In [3]:
model_url = "https://www.kaggle.com/models/tensorflow/ssd-mobilenet-v2/frameworks/TensorFlow2/variations/ssd-mobilenet-v2/versions/1"
model = hub.load(model_url)

In [4]:
n = 1000
sampled_dataset = dataset.take(n)

In [9]:
@track_emissions()
def inference(sampled_dataset):
    coco_results = []
    print("_______________________________________________________________")
    print("Doing inference for", len(sampled_dataset), "images using the full pre-trained ssd mobilnet v2...")
    start_time = time.time()
    for sample in sampled_dataset:
        coco_sample_results = []

        image_path = sample.filepath
        image = tf.image.decode_image(tf.io.read_file(image_path), channels=3).numpy()
        image_height, image_width, _ = image.shape
        image_id = int(image_path[-16:-4])
        input_image = tf.convert_to_tensor(image)
        input_image = tf.expand_dims(input_image, axis=0)
        input_image = tf.cast(input_image, tf.uint8) 

        detections = model(input_image)

        boxes = detections["detection_boxes"][0].numpy()
        scores = detections["detection_scores"][0].numpy()
        labels = detections["detection_classes"][0].numpy().astype(int)

        for box, score, label in zip(boxes, scores, labels):
            coco_sample_results.append(
                {
                    "image_id": image_id,
                    "category_id": label,
                    "bbox":  [
                        box[1] * image_width,
                        box[0] * image_height,
                        (box[3] - box[1]) * image_width,
                        (box[2] - box[0]) * image_height,
                    ],
                    "score": score,
                }
            )

        coco_results.extend(coco_sample_results)

    end_time = time.time()
    
    inferencetime = end_time - start_time
    print("Inference time:", inferencetime)
    print("___________________________________________________________")
    return coco_results

In [11]:
coco_results = inference(dataset)

[codecarbon INFO @ 09:49:07] [setup] RAM Tracking...
[codecarbon INFO @ 09:49:07] [setup] GPU Tracking...
[codecarbon INFO @ 09:49:07] No GPU found.
[codecarbon INFO @ 09:49:07] [setup] CPU Tracking...
[codecarbon WARNING @ 09:49:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:49:07] We saw that you have a Apple M2 but we don't know it. Please contact us.
[codecarbon INFO @ 09:49:07] CPU Model on constant consumption mode: Apple M2
[codecarbon INFO @ 09:49:07] >>> Tracker's metadata:
[codecarbon INFO @ 09:49:07]   Platform system: macOS-13.5-arm64-arm-64bit
[codecarbon INFO @ 09:49:07]   Python version: 3.11.0
[codecarbon INFO @ 09:49:07]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 09:49:07]   Available RAM : 16.000 GB
[codecarbon INFO @ 09:49:07]   CPU count: 8
[codecarbon INFO @ 09:49:07]   CPU model: Apple M2
[codecarbon INFO @ 09:49:07]   GPU count: None
[codecarbon INFO @ 09:49:07]   GPU model: None


_______________________________________________________________
Doing inference for 5000 images using the full pre-trained ssd mobilnet v2...


[codecarbon INFO @ 09:49:25] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 09:49:25] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:49:25] 0.000202 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:49:40] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 09:49:40] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:49:40] 0.000404 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:49:55] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 09:49:55] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:49:55] 0.000606 kWh of electricity used since the beginning.
[codecarbon INFO @ 09:50:10] Energy consumed for RAM : 0.000100 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 09:50:10] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codeca

Inference time: 215.36327505111694
___________________________________________________________


In [12]:
print("Saving results to json file...")
coco_results_path = "coco_results.json"
with open(coco_results_path, "w") as f:
    json.dump(coco_results, f, cls=NumpyEncoder)
print("coco_results.json created & saved!")

print("___________________________________________________________")
# Load ground truth annotations
gt_path = 'data/coco/annotations/instances_val2017.json'
coco_gt = COCO(gt_path)

# Our prediction results
coco_dt = coco_gt.loadRes(coco_results_path)
print("___________________________________________________________")
print("Evaluating the model...")
coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()
print("___________________________________________________________")

Saving results to json file...
coco_results.json created & saved!
___________________________________________________________
loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.14s)
creating index...
index created!
___________________________________________________________
Evaluating the model...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=16.31s).
Accumulating evaluation results...
DONE (t=4.20s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.349
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.204
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.027
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.414
 Average Recal

### TfLite


In [13]:
representative_dataset = dataset.take(500)

In [1]:
def preprocess_image(image):
    input_image = tf.convert_to_tensor(image)
    input_image = tf.expand_dims(input_image, axis=0)
    input_image = tf.cast(input_image, tf.uint8) 
    image = tf.image.resize(image, (320, 320)) 
    image = tf.cast(image, tf.uint8)
    return input_image

def representative_data_gen():
    for sample in representative_dataset:
        image_path = sample.filepath        
        image = tf.image.decode_image(tf.io.read_file(image_path), channels=3)
        image = preprocess_image(image) 
        yield [image]

#### int8 quantization

In [15]:
from tensorflow import lite

# int8 quantization
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS]


converter.representative_dataset = representative_data_gen 

tflite_model_int8 = converter.convert()

print("Saving the model...")
with open("model_int8.tflite", 'wb') as f:
    f.write(tflite_model_int8)
print("model_int8.tflite created & saved!")
print("___________________________________________________________")

INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmpjmgloeay/assets


INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmpjmgloeay/assets
/Users/kristian/anaconda3/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-11-20 09:54:48.135050: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 09:54:48.135067: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 272, Total Ops 2004, % non-converted = 13.57 %
 * 272 ARITH ops

- arith.constant:  272 occurrences  (f32: 161, i32: 111)



  (f32: 15)
  (f32: 3, i1: 1, i32: 3)
  (f32: 98, i32: 90)
  (f32: 55)
  (f32: 17)
  (f32: 2)
  (f32: 1)
  (f32: 91, i32: 90)
  (f32: 295)
  (i1: 7)
  (i1: 1)
  (i1: 90)
  (f

Saving the model...
model_int8.tflite created & saved!
___________________________________________________________


#### Float16 quantization

In [16]:
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

converter.representative_dataset = representative_data_gen

tflite_model_fp16 = converter.convert()
print("Saving the model...")
with open("model_fp16.tflite", 'wb') as f:
    f.write(tflite_model_fp16)
print("model_fp16.tflite created & saved!")
print("___________________________________________________________")


INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmp_2e5v0s2/assets


INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmp_2e5v0s2/assets
2023-11-20 10:00:47.684993: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 10:00:47.685007: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 271, Total Ops 2164, % non-converted = 12.52 %
 * 271 ARITH ops

- arith.constant:  271 occurrences  (f16: 160, i32: 111)



  (f32: 15)
  (f32: 3, i1: 1, i32: 3)
  (f32: 98, i32: 90)
  (f32: 55)
  (f32: 17)
  (f32: 161)
  (f32: 2)
  (f32: 1)
  (f32: 91, i32: 90)
  (f32: 295)
  (i1: 7)
  (i1: 1)
  (i1: 90)
  (f32: 1)
  (f32: 4, i32: 1)
  (f32: 14)
  (i32: 90)
  (f32: 6, i32: 9)
  (f32: 5)
  (i32: 2)
  (f32: 4)
  (i64: 1, f32: 106, i1: 1, i32: 98)
  (f32: 1)
  (f32: 91, i32: 6)
  (i32: 103)
  (f32: 96)
 

Saving the model...
model_fp16.tflite created & saved!
___________________________________________________________


#### Float32 quantization

In [17]:
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float32]

converter.representative_dataset = representative_data_gen

tflite_model_fp32 = converter.convert()

print("Saving the model...")
with open("model_fp32.tflite", 'wb') as f:
    f.write(tflite_model_fp32)
print("model_fp32.tflite created & saved!")
print("___________________________________________________________")

INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmpjwxe65gm/assets


INFO:tensorflow:Assets written to: /var/folders/8v/5_c7kn_13jsgmgqz56r3yx440000gp/T/tmpjwxe65gm/assets
2023-11-20 10:00:59.238302: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 10:00:59.238312: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 272, Total Ops 2004, % non-converted = 13.57 %
 * 272 ARITH ops

- arith.constant:  272 occurrences  (f32: 161, i32: 111)



  (f32: 15)
  (f32: 3, i1: 1, i32: 3)
  (f32: 98, i32: 90)
  (f32: 55)
  (f32: 17)
  (f32: 2)
  (f32: 1)
  (f32: 91, i32: 90)
  (f32: 295)
  (i1: 7)
  (i1: 1)
  (i1: 90)
  (f32: 1)
  (f32: 4, i32: 1)
  (f32: 14)
  (i32: 90)
  (f32: 6, i32: 9)
  (f32: 5)
  (i32: 2)
  (f32: 4)
  (i64: 1, f32: 106, i1: 1, i32: 98)
  (f32: 1)
  (f32: 91, i32: 6)
  (i32: 103)
  (f32: 96)
  (f32: 4)
  (

Saving the model...
model_fp32.tflite created & saved!
___________________________________________________________


In [18]:
import cv2
def setup_model(model_content):
    print("___________________________________________________________")
    print("Setting up the model...")
    interpreter = tf.lite.Interpreter(model_path=model_content)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = (1, 320, 320, 3)  
    interpreter.resize_tensor_input(input_details[0]['index'], input_shape)
    interpreter.resize_tensor_input(output_details[0]['index'], input_shape)

    interpreter.allocate_tensors()

    return interpreter, input_details, output_details


In [26]:
def scale_image(image_path, size=320):
    image = cv2.imread(image_path)

    height, width, channels = image.shape

    if height > width:
        new_height = size
        scale = size / height
        new_width = int(scale * width)
    else:
        new_width = size
        scale = size / width
        new_height = int(scale * height)

    resized_image = cv2.resize(image, (new_width, new_height))

    padded_image = np.zeros((size, size, channels), dtype=np.uint8)
    top = (size - new_height) // 2
    left = (size - new_width) // 2
    padded_image[top:top + new_height, left:left + new_width] = resized_image

    input_tensor = tf.convert_to_tensor(padded_image)
    input_tensor = input_tensor[tf.newaxis, ...]

    return input_tensor, new_height, new_width, height, width

def rescale_box(image_id, detection_boxes, detection_classes, detection_scores, new_height, new_width, height, width, size=320):
    all_results = []

    box = np.round(detection_boxes[0] * size).astype(int)

    height_scale = height / new_height
    width_scale = width / new_width

    top_padding = (size - new_height) // 2
    left_padding = (size - new_width) // 2

    for i in range(len(detection_boxes[0])):
        ystart, xstart, yend, xend = box[i]

        ystart = (ystart - top_padding) * height_scale
        xstart = (xstart - left_padding) * width_scale
        yend = (yend - top_padding) * height_scale
        xend = (xend - left_padding) * width_scale

        box_width = xend - xstart
        box_height = yend - ystart

        result_image = {
            "image_id": int(image_id),
            "category_id": int(detection_classes[0][i]),
            "bbox": [xstart, ystart, box_width, box_height],
            "score": float(detection_scores[0][i])
        }

        all_results.append(result_image)

    return all_results

@track_emissions()
def inference(sampled_dataset, interpreter, input_details, output_details):
    coco_results = []

    for sample in sampled_dataset:
        image_path = sample.filepath
        image_id = int(image_path[-16:-4])

        input_image, new_height, new_width, height, width = scale_image(image_path)
        boxes, classes, scores = test_model(input_image, interpreter, input_details, output_details)
        predictions = rescale_box(image_id, boxes, classes, scores, new_height, new_width, height, width)

        coco_results.extend(predictions)

    return coco_results

def test_model(input_tensor, interpreter, input_details, output_details):
    interpreter.set_tensor(input_details[0]['index'], input_tensor)

    interpreter.invoke()

    output_details = interpreter.get_output_details()

    detection_boxes = interpreter.get_tensor(output_details[4]['index'])
    detection_classes = interpreter.get_tensor(output_details[5]['index'])
    detection_scores = interpreter.get_tensor(output_details[6]['index'])

    return detection_boxes, detection_classes, detection_scores


def evaluate(coco_results):
    coco_results_path = "coco_results.json"
    with open(coco_results_path, "w") as f:
        json.dump(coco_results, f, cls=NumpyEncoder)

    gt_path = 'data/coco/annotations/instances_val2017.json'
    coco_gt = COCO(gt_path)

    coco_dt = coco_gt.loadRes(coco_results_path)

    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[0]

In [23]:
models = ["model_int8.tflite", "model_fp16.tflite", "model_fp32.tflite"]

In [27]:
sampled_dataset = dataset

In [29]:
for model in models:
    print("Running model: ", model)
    interpreter, input_details, output_details = setup_model(model)
    start_time = time.time()
    coco_results = inference(sampled_dataset, interpreter, input_details, output_details)
    end_time = time.time()
    print("Inference time: ", end_time - start_time)
    evaluation = evaluate(coco_results)
    print("______________________________________________________")

[codecarbon INFO @ 10:20:05] [setup] RAM Tracking...
[codecarbon INFO @ 10:20:05] [setup] GPU Tracking...
[codecarbon INFO @ 10:20:05] No GPU found.
[codecarbon INFO @ 10:20:05] [setup] CPU Tracking...
[codecarbon WARNING @ 10:20:05] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:20:05] We saw that you have a Apple M2 but we don't know it. Please contact us.
[codecarbon INFO @ 10:20:05] CPU Model on constant consumption mode: Apple M2
[codecarbon INFO @ 10:20:05] >>> Tracker's metadata:
[codecarbon INFO @ 10:20:05]   Platform system: macOS-13.5-arm64-arm-64bit
[codecarbon INFO @ 10:20:05]   Python version: 3.11.0
[codecarbon INFO @ 10:20:05]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 10:20:05]   Available RAM : 16.000 GB
[codecarbon INFO @ 10:20:05]   CPU count: 8
[codecarbon INFO @ 10:20:05]   CPU model: Apple M2
[codecarbon INFO @ 10:20:05]   GPU count: None
[codecarbon INFO @ 10:20:05]   GPU model: None


Running model:  model_int8.tflite
___________________________________________________________
Setting up the model...


[codecarbon INFO @ 10:20:20] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:20:20] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:20:20] 0.000202 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:20:35] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:20:35] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:20:35] 0.000404 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:20:50] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:20:50] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:20:50] 0.000606 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:21:05] Energy consumed for RAM : 0.000100 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:21:05] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codeca

Inference time:  205.02662086486816
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.71s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.53s).
Accumulating evaluation results...
DONE (t=4.27s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.076
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.168
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.102
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.161
 Average Recall     (AR) @[ 

[codecarbon INFO @ 10:23:54] [setup] RAM Tracking...
[codecarbon INFO @ 10:23:54] [setup] GPU Tracking...
[codecarbon INFO @ 10:23:54] No GPU found.
[codecarbon INFO @ 10:23:54] [setup] CPU Tracking...
[codecarbon WARNING @ 10:23:54] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:23:54] We saw that you have a Apple M2 but we don't know it. Please contact us.
[codecarbon INFO @ 10:23:54] CPU Model on constant consumption mode: Apple M2
[codecarbon INFO @ 10:23:54] >>> Tracker's metadata:
[codecarbon INFO @ 10:23:54]   Platform system: macOS-13.5-arm64-arm-64bit
[codecarbon INFO @ 10:23:54]   Python version: 3.11.0
[codecarbon INFO @ 10:23:54]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 10:23:54]   Available RAM : 16.000 GB
[codecarbon INFO @ 10:23:54]   CPU count: 8
[codecarbon INFO @ 10:23:54]   CPU model: Apple M2
[codecarbon INFO @ 10:23:54]   GPU count: None
[codecarbon INFO @ 10:23:54]   GPU model: None


...

Inference time:  187.19544315338135
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.79s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.67s).
Accumulating evaluation results...
DONE (t=4.31s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.336
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.178
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.268
 Average Recall     (AR) @[ 

[codecarbon INFO @ 10:27:26] [setup] RAM Tracking...
[codecarbon INFO @ 10:27:26] [setup] GPU Tracking...
[codecarbon INFO @ 10:27:26] No GPU found.
[codecarbon INFO @ 10:27:26] [setup] CPU Tracking...
[codecarbon WARNING @ 10:27:26] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 10:27:26] We saw that you have a Apple M2 but we don't know it. Please contact us.
[codecarbon INFO @ 10:27:26] CPU Model on constant consumption mode: Apple M2
[codecarbon INFO @ 10:27:26] >>> Tracker's metadata:
[codecarbon INFO @ 10:27:26]   Platform system: macOS-13.5-arm64-arm-64bit
[codecarbon INFO @ 10:27:26]   Python version: 3.11.0
[codecarbon INFO @ 10:27:26]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 10:27:26]   Available RAM : 16.000 GB
[codecarbon INFO @ 10:27:26]   CPU count: 8
[codecarbon INFO @ 10:27:26]   CPU model: Apple M2
[codecarbon INFO @ 10:27:26]   GPU count: None
[codecarbon INFO @ 10:27:26]   GPU model: None


______________________________________________________
Running model:  model_fp32.tflite
___________________________________________________________
Setting up the model...


[codecarbon INFO @ 10:27:42] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:27:42] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:27:42] 0.000202 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:27:57] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:27:57] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:27:57] 0.000404 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:28:12] Energy consumed for RAM : 0.000075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:28:12] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 10:28:12] 0.000606 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:28:27] Energy consumed for RAM : 0.000100 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 10:28:27] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codeca

Inference time:  184.5064160823822
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.76s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.85s).
Accumulating evaluation results...
DONE (t=4.35s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.149
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.258
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.108
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.335
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.178
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.269
 Average Recall     (AR) @[ I